In [1]:
import math
import os
import sys

REPO_ROOT = os.environ["REPO_PATH"]
PROJECT_NAME = "advent-of-code-2023"
sys.path.append(os.path.join(REPO_ROOT, PROJECT_NAME))

import utils

# Configs.

In [2]:
DIRECTIONS = {
    "|": {"end1": [-1, 0], "end2": [1, 0]},
    "-": {"end1": [0, -1], "end2": [0, 1]},
    "L": {"end1": [-1, 0], "end2": [0, 1]},
    "J": {"end1": [-1, 0], "end2": [0, -1]},
    "7": {"end1": [1, 0], "end2": [0, -1]},
    "F": {"end1": [1, 0], "end2": [0, 1]},
}

# Functions

In [3]:
class EndOfPath(Exception):
    pass


class Pipe:
    def __init__(self, position: list[int], pipe_type: str):
        self.position = position
        self.pipe_type = pipe_type
        self.direction = DIRECTIONS.get(pipe_type, {"end1": [0, 0], "end2": [0, 0]})
        self.end1 = [self.position[0] + self.direction["end1"][0], self.position[1] + self.direction["end1"][1]]
        self.end2 = [self.position[0] + self.direction["end2"][0], self.position[1] + self.direction["end2"][1]]
        if self.end1[0] < 0 or self.end1[1] < 0:
            self.end1 = None
        if self.end2[0] < 0 or self.end2[1] < 0:
            self.end2 = None
        self.upstream_end = None
        self.downstream_end = None
        self.position_in_chain = 0

    def __find_neighbor(self, pipe_map: list[list[str]]) -> object:
        directions = [[1, 0], [-1, 0], [0, 1], [0, -1]]
        for direction in directions:
            end1 = [self.position[0] + direction[0], self.position[1] + direction[1]]
            if end1[0] < 0 or end1[1] < 0:
                continue
            neighbor = pipe_map[end1[0]][end1[1]]
            self.end1 = end1
            if neighbor.end2 == self.position or neighbor.end1 == self.position:
                return neighbor
        raise Exception("No neighbor found")

    def connect_neighbor(self, pipe_map: list[list[object]]) -> object:
        if self.upstream_end == self.end1:
            neighbor = pipe_map[self.end2[0]][self.end2[1]]
            self.downstream_end = self.end2
        elif self.upstream_end == self.end2:
            neighbor = pipe_map[self.end1[0]][self.end1[1]]
            self.downstream_end = self.end1
        elif self.pipe_type == "S":
            neighbor = self.__find_neighbor(pipe_map)
            self.downstream_end = self.end1
        else:
            raise Exception("Pipe not connected to upstream")

        if neighbor.pipe_type == "S":
            raise EndOfPath("End of path reached")

        if neighbor.end2 == self.position:
            neighbor.upstream_end = neighbor.end2
        elif neighbor.end1 == self.position:
            neighbor.upstream_end = neighbor.end1
        else:
            raise Exception("Neighbor not connectable")

        neighbor.position_in_chain = self.position_in_chain + 1
        return neighbor

    def __repr__(self):
        return f"Pipe({self.position=}, {self.pipe_type=}, {self.end1=}, {self.end2=}, {self.upstream_end=}, {self.position_in_chain=})"

# Extract data

In [4]:
puzzle_data = utils.load_data(test=False)
puzzle_data = [list(line) for line in puzzle_data]

# Puzzle 1

In [5]:
pipe_map = []
start_coord = None
for r, row in enumerate(puzzle_data):
    pipe_col = []
    for c, col in enumerate(row):
        pipe_col.append(Pipe([r, c], col))
        if col == "S":
            start_coord = [r, c]
    pipe_map.append(pipe_col)

In [6]:
node = pipe_map[start_coord[0]][start_coord[1]]
while True:
    try:
        node = node.connect_neighbor(pipe_map)
    except EndOfPath:
        break
farthest_point = math.ceil(node.position_in_chain / 2)
print(f"{farthest_point=}")

farthest_point=7066


# Puzzle 2